In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
import json

In [54]:
search_result = lk.search_lightcurve("KIC 10874614", mission="Kepler")
lc = search_result[0].download()

lc = lc.remove_nans().normalize()
lc_flat = lc.flatten(window_length=401)


In [55]:
period = 3.2347  # days (Kepler-6b)
lc_folded = lc_flat.fold(period=period)


In [56]:
depth = 1 - np.min(lc_folded.flux)
print(f"Transit depth: {depth:.5f}")


Transit depth: 0.01220


In [57]:
threshold = 1 - depth / 2
in_transit = lc_folded.flux < threshold

phase = lc_folded.phase[in_transit]
duration_phase = phase.max() - phase.min()
duration_days = (duration_phase * period).value

print(f"Transit duration: {duration_days:.3f} days")


Transit duration: 0.412 days


In [58]:
out_of_transit = lc_folded.flux[~in_transit]

noise = np.std(out_of_transit)
snr = depth / noise

print(f"Transit SNR: {snr:.2f}")


Transit SNR: 16.57


# Extracted Transit Features (Kepler-6b)
The following quantities were computed from the phase-folded, flattened light curve. Astropy quantities were converted to floats for downstream analysis and reproducibility.

In [59]:
features = {
    "Period (Days)": float(period),
    "Depth": float(depth),
    "Duration (Days)": float(duration_days),
    "SNR": float(snr)
}

features


{'Period (Days)': 3.2347,
 'Depth': 0.012197920474950497,
 'Duration (Days)': 0.41246971906421087,
 'SNR': 16.57099421142315}

In [ ]:

with open("../data/features_kepler6b.json", "w") as f:
    json.dump(features, f, indent=2)